In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#!git clone https://huggingface.co/spaces/nguyenvulebinh/spoken-norm-taggen-v2

Cloning into 'spoken-norm-taggen-v2'...
remote: Repository not found
fatal: repository 'https://huggingface.co/spaces/nguyenvulebinh/spoken-norm-taggen-v2/' not found


In [ ]:
#!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.6/553.6 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.3/421.3 kB 7.2 MB/s eta 0:00:00
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-linux_x86_64.whl size=3184349 sha256=4ef924a18ee47a1f7b8733fb6e2853c91078575dc59eaa3860fc44ccdb6d5226
  Stored in directory: /tmp/pip-ephem-wheel-cache-63jh9_he/wheels/a5/73/ee/670fbd0cee8f6f0b21d10987cb042291e662e26e1a07026462
Successfully built kenlm


In [ ]:
%cd /content/drive/MyDrive/train_data

/content/drive/MyDrive/train_data


In [ ]:
#!pip install -r requirements.txt

In [ ]:
#!pip install -q sentencepiece regtag validators gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.1 MB/s eta 0:00:0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict, ClassLabel, concatenate_datasets
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML
import re
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2ProcessorWithLM, Wav2Vec2FeatureExtractor,Wav2Vec2ForCTC, TrainingArguments, Trainer
import json
import soundfile as sf
import os
import pickle
import IPython.display as ipd
from audiomentations import Lambda, Compose, AddGaussianNoise, AddGaussianSNR, TimeStretch, PitchShift, Shift, AddBackgroundNoise, OneOf
import colorednoise as cn
import torch
# import kenlm
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from huggingface_hub import hf_hub_download
from importlib.machinery import SourceFileLoader
# from pyctcdecode import Alphabet, BeamSearchDecoderCTC, LanguageModel, build_ctcdecoder

In [ ]:
%cd /content/drive/MyDrive/train_data

/content/drive/MyDrive/train_data


In [ ]:
df = pd.read_json("train_20230909.jsonl", lines = True)
data = Dataset.from_pandas(df)

In [ ]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(data)

,id,sentence,intent,sentence_annotation,entities,file
0,64871f1a1ae676a65cb64e9b,bạn ơi mở giúp mình cái máy hút bụi với ở chỗ nhà bếp ý,mở thiết bị,bạn ơi [ command : mở ] giúp mình cái [ device : máy hút bụi ] với ở chỗ [ location : nhà bếp ] ý,"[{'filler': 'mở', 'type': 'command'}, {'filler': 'máy hút bụi', 'type': 'device'}, {'filler': 'nhà bếp', 'type': 'location'}]",64871f1a1ae676a65cb64e9b.wav
1,648de1024c840264703e3cf4,"cái đèn led kia có vấn ề rồi, kiểm tra cho tôi nhá",kiểm tra tình trạng thiết bị,"cái [ device : đèn led ] kia có vấn ề rồi, [ command : kiểm tra ] cho tôi nhá","[{'filler': 'đèn led', 'type': 'device'}, {'filler': 'kiểm tra', 'type': 'command'}]",648de1024c840264703e3cf4.wav
2,648c7a48f73a57ee107d22dc,mình không dùng nữa rồi đóng cho mình cái van nước ở trong nhà tắm với,đóng thiết bị,mình không dùng nữa rồi [ command : đóng ] cho mình cái [ device : van nước ] ở trong [ location : nhà tắm ] với,"[{'filler': 'đóng', 'type': 'command'}, {'filler': 'van nước', 'type': 'device'}, {'filler': 'nhà tắm', 'type': 'location'}]",648c7a48f73a57ee107d22dc.wav
3,648c94e0f73a571ec57d7682,kiểm tra cho tôi cái đèn nhé,kiểm tra tình trạng thiết bị,[ command : kiểm tra ] cho tôi cái [ device : đèn ] nhé,"[{'filler': 'kiểm tra', 'type': 'command'}, {'filler': 'đèn', 'type': 'device'}]",648c94e0f73a571ec57d7682.wav
4,64874a6e1ae6766f4eb69485,em ơi giảm cho anh cái bóng chùm xuống 85% với,giảm độ sáng của thiết bị,em ơi [ command : giảm ] cho anh cái [ device : bóng chùm ] xuống [ target number : 85% ] với,"[{'filler': 'giảm', 'type': 'command'}, {'filler': 'bóng chùm', 'type': 'device'}, {'filler': '85%', 'type': 'target number'}]",64874a6e1ae6766f4eb69485.wav


In [ ]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\\/"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
    return batch

In [ ]:
data = data.map(remove_special_characters)

Map:   0%|          | 0/7490 [00:00<?, ? examples/s]

In [ ]:
show_random_elements(data, num_examples=5)

,id,sentence,intent,sentence_annotation,entities,file,text
0,6487446d1ae6765c42b68ce8,em ơi lát bật cho anh cái máy fax với,bật thiết bị,em ơi lát [ command : bật ] cho anh cái [ device : máy fax ] với,"[{'filler': 'bật', 'type': 'command'}, {'filler': 'máy fax', 'type': 'device'}]",6487446d1ae6765c42b68ce8.wav,em ơi lát bật cho anh cái máy fax với
1,64872f8f1ae676f437b66c04,em ơi bật anh cái đèn bàn lên với tối thế,bật thiết bị,em ơi [ command : bật ] anh cái [ device : đèn bàn ] lên với tối thế,"[{'filler': 'bật', 'type': 'command'}, {'filler': 'đèn bàn', 'type': 'device'}]",64872f8f1ae676f437b66c04.wav,em ơi bật anh cái đèn bàn lên với tối thế
2,648dc7884c84024b2a3e0e9f,tắt giúp anh cái đèn chùm với,tắt thiết bị,[ command : tắt ] giúp anh cái [ device : đèn chùm ] với,"[{'filler': 'tắt', 'type': 'command'}, {'filler': 'đèn chùm', 'type': 'device'}]",648dc7884c84024b2a3e0e9f.wav,tắt giúp anh cái đèn chùm với
3,648f03bebd5f01a1b4bbb5a4,làm ơn hãy đóng lò nướng lại,đóng thiết bị,làm ơn hãy [ command : đóng ] [ device : lò nướng ] lại,"[{'filler': 'đóng', 'type': 'command'}, {'filler': 'lò nướng', 'type': 'device'}]",648f03bebd5f01a1b4bbb5a4.wav,làm ơn hãy đóng lò nướng lại
4,64849a6819e0403c495e5542,em ơi mở cho chị cái máy hút ẩm,mở thiết bị,em ơi [ command : mở ] cho chị cái [ device : máy hút ẩm ],"[{'filler': 'mở', 'type': 'command'}, {'filler': 'máy hút ẩm', 'type': 'device'}]",64849a6819e0403c495e5542.wav,em ơi mở cho chị cái máy hút ẩm


In [ ]:
nums = [str(i) for i in range(10)]
nums

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [ ]:
def not_contain_nums(batch):
  for char in nums:
    if char in batch['text']:
      return False
  return True

In [ ]:
data_no_num = data.filter(not_contain_nums)

Filter:   0%|          | 0/7490 [00:00<?, ? examples/s]

In [ ]:
len(data_no_num)

5761

In [ ]:
processor = Wav2Vec2ProcessorWithLM.from_pretrained("nguyenvulebinh/wav2vec2-base-vi-vlsp2020")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

#Don't run cause we can't fix it

In [ ]:
# def extract_all_chars(batch):
#   all_text = " ".join(batch["text"])
#   vocab = list(set(all_text))
#   return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
# vocab = data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=data.column_names['train'])

In [ ]:
# vocab_list = list(set(vocab['train']["vocab"][0]) |
#                   set(pretrained_vocab.keys()) - {"<s>", "</s>", "<pad>", "<unk>", "|", "j"})
# vocab_dict = {v: k for k, v in enumerate(vocab_list)}
# vocab_dict["|"] = vocab_dict[" "]
# del vocab_dict[" "]
# vocab_dict["[UNK]"] = len(vocab_dict)
# vocab_dict["[PAD]"] = len(vocab_dict)

In [ ]:
# with open('/kaggle/working/vocab.json', 'w') as vocab_file:
#     json.dump(vocab_dict, vocab_file)

In [ ]:
# tokenizer = Wav2Vec2CTCTokenizer("/kaggle/working/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
# processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

NameError: name 'feature_extractor' is not defined

In [ ]:
# processor.tokenizer = tokenizer

In [ ]:
# len(processor.tokenizer)

#OK now continue

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [ ]:
%cd /train_data/Train

/content/drive/MyDrive/train_data/Train


In [ ]:
data = data.map(speech_file_to_array_fn, remove_columns=data.column_names)

Map:   0%|          | 0/7490 [00:00<?, ? examples/s]

In [ ]:
import gc

gc.collect()

177

In [ ]:
import sys

sys.getsizeof(processor)

48

In [ ]:
rand_int = random.randint(0, len(data))

ipd.Audio(data=np.asarray(data[rand_int]["speech"]), autoplay=True, rate=16000)

In [ ]:
data = data.train_test_split(test_size=0.2, seed=42)

In [ ]:
print("Target text:", data['train'][rand_int]["target_text"])
print("Input array shape:", np.asarray(data['train'][rand_int]["speech"]).shape)
print("Sampling rate:", data['train'][rand_int]["sampling_rate"])

KeyError: ignored

In [ ]:
def PinkNoiseSNR(samples, sample_rate, min_snr=5.0, max_snr=15.0):
    snr = np.random.uniform(min_snr, max_snr)
    a_signal = np.sqrt(samples ** 2).max()
    a_noise = a_signal / (10 ** (snr / 20))

    pink_noise = cn.powerlaw_psd_gaussian(1, len(samples))
    a_pink = np.sqrt(pink_noise ** 2).max()
    samples = (samples + pink_noise * 1 / a_pink * a_noise)
    return samples.astype(np.float32)

def VolumeControl(samples, sample_rate, mode="sine", db_limit=10.0):
    db = np.random.uniform(-db_limit, db_limit)
    if mode == "uniform":
        db_translated = 10 ** (db / 20)
    elif mode == "fade":
        lin = np.arange(len(samples))[::-1] / (len(samples) - 1)
        db_translated = 10 ** (db * lin / 20)
    elif mode == "cosine":
        cosine = np.cos(np.arange(len(samples)) / len(samples) * np.pi * 2)
        db_translated = 10 ** (db * cosine / 20)
    else:
        sine = np.sin(np.arange(len(samples)) / len(samples) * np.pi * 2)
        db_translated = 10 ** (db * sine / 20)
    db_translated
    augmented = (samples * db_translated)
    return augmented.astype(np.float32)

def augment(batch):
    aug = Compose([
        OneOf(
            transforms=[Lambda(transform=PinkNoiseSNR, p=0.5),
                        AddGaussianSNR(min_snr_db=5.0, max_snr_db=15.0, p=0.5)]
        ),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
#         PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
        Lambda(transform=VolumeControl, p=0.5),
    ])
    return {"speech": aug(np.asarray(batch["speech"], dtype=(np.float32)), sample_rate=16000)}

In [ ]:
orig_size = len(data['train'])
augment_size = int(orig_size * 0.2)
random_idxs = np.random.choice(orig_size, augment_size, replace=False)
data_subset = data['train'].select(random_idxs)
augmented = data_subset.map(augment)

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

In [ ]:
#combined = concatenate_datasets([data['train'], augmented])
data['train'] = concatenate_datasets([data['train'], augmented]).shuffle(seed = 42)
del augment

In [ ]:
import gc

gc.collect()

82

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

    # input_values = processor(
    #     batch["speech"],
    #     sampling_rate=batch["sampling_rate"],
    #     return_tensors="pt"
    # ).input_values.to("cuda")

    # with torch.no_grad():
    #   logits = model(input_values).logits

    # pred_ids = torch.argmax(logits, dim=-1)
    # batch["pred_str"] = processor.batch_decode(pred_ids)[0]

In [ ]:
#data_prepared = data.map(prepare_dataset, remove_columns=data.column_names['train'], batch_size=8, batched=True)

Map:   0%|          | 0/5529 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:639: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1153 [00:00<?, ? examples/s]

In [ ]:
%cd /content/drive/MyDrive/train_data

/content/drive/MyDrive/train_data


In [ ]:
import pickle

with open('data_prepared.dat', 'rb') as f:
  data_prepared = pickle.load(f)

In [ ]:
data_prepared

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 5529
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1153
    })
})

In [ ]:
%cd /content/drive/MyDrive/SLU_hackathon/SLU_dataset

/root


In [ ]:
if not (os.path.isfile('data.dat')):
    with open('data.dat', 'wb') as f:
        pickle.dump(data, f)
else:
    with open('data.dat', 'rb') as f:
        data = pickle.load(f)
data

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 5243
    })
    valid: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 1498
    })
    test: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 749
    })
})

In [ ]:
del data_no_num
gc.collect()

NameError: ignored

In [ ]:
# if not (os.path.isfile('data_prepared.dat')):
# with open('data_prepared.dat', 'wb') as f:
#     pickle.dump(data_prepared, f)
# else:
with open('data_prepared.dat', 'rb') as f:
    data_prepared = pickle.load(f)

In [ ]:
data_prepared['train'] = data_prepared['train'].shuffle(seed = 42)

In [ ]:
# with open('data.dat', 'wb') as f:
#     pickle.dump(data_prepared, f)

# with open('data_prepared.dat', 'wb') as f:
#     pickle.dump(data_prepared, f)

In [ ]:
from evaluate import load
wer_metric = load("wer")

In [ ]:
len(processor.tokenizer)

98

In [ ]:
# model = SourceFileLoader("model", hf_hub_download(repo_id="nguyenvulebinh/wav2vec2-base-vi-vlsp2020", filename="model_handling.py")).load_module().Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vi-vlsp2020")
model = SourceFileLoader("model", hf_hub_download(repo_id="nguyenvulebinh/wav2vec2-base-vi-vlsp2020", filename="model_handling.py")).load_module().Wav2Vec2ForCTC.from_pretrained(
    "Chiizu/wav2vec2-base-vi-vlsp2020-demo",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
    ignore_mismatched_sizes=True,
)
model.to('cuda')

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2ProcessorWithLM
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
%cd /content/spoken-norm-taggen

/content/spoken-norm-taggen


In [ ]:
from infer import infer

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    if type(processor).__name__ == "Wav2Vec2ProcessorWithLM":
        pred_str = processor.batch_decode(pred_logits).text
    else:
        pred_ids = np.argmax(pred_logits, axis=-1)
        pred_str = infer(processor.batch_decode(pred_ids), [])

    # this will always work (even if the processor has a decoder LM)
    label_str = processor.tokenizer.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
    output_dir="wav2vec2-base-vi-vlsp2020-demo",
    group_by_length=True,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    num_train_epochs=3,
    fp16=True,
#     save_steps=500,
#     eval_steps=10,
    #logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.005,
    #warmup_steps=1000,
    save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# from torch import nn
# from transformers import Trainer


# class CustomTrainer(Trainer):
#     def compute_loss(self, model, inputs, return_outputs=False):
#         labels = inputs.pop("labels")
#         # forward pass
#         outputs = model(**inputs)
#         logits = outputs.get("logits")
#         # compute custom loss (suppose one has 3 labels with different weights)
#         loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
#         loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
#         return (loss, outputs) if return_outputs else loss

In [ ]:
!pip install --upgrade accelerator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.5 MB/s eta 0:00:00


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data_prepared['train'],
    eval_dataset=data_prepared['test'],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:639: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Wer
1,0.724500,0.278998,0.110095
2,0.574600,0.246682,0.091895
3,0.406800,0.229199,0.083967


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:639: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2_with_lm/processing_wav2vec2_with_lm.py:639: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


TrainOutput(global_step=2076, training_loss=0.533508573653381, metrics={'train_runtime': 1211.2359, 'train_samples_per_second': 13.694, 'train_steps_per_second': 1.714, 'total_flos': 5.890496629218159e+17, 'train_loss': 0.533508573653381, 'epoch': 3.0})

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/385M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

'https://huggingface.co/Chiizu/wav2vec2-base-vi-vlsp2020-demo/tree/main/'

In [ ]:
del model

In [ ]:
model = SourceFileLoader("model", hf_hub_download(repo_id="nguyenvulebinh/wav2vec2-base-vi-vlsp2020", filename="model_handling.py")).load_module().Wav2Vec2ForCTC.from_pretrained("anhtu77/wav2vec2-base-vi-vlsp2020-demo")
model.to("cuda")

In [ ]:
%cd /content/spoken-norm-taggen/

/content/spoken-norm-taggen


In [ ]:
import infer
from infer import infer

In [ ]:
bias_list = []

In [ ]:
def map_to_result(batch):
    # model.to("cuda")

    input_data = processor.feature_extractor(batch["speech"], sampling_rate=batch["sampling_rate"], return_tensors='pt')
    for key, val in input_data.items():
        input_data[key] = val.cuda()

    output = model(**input_data)

    batch["pred_str"] = processor.decode(output.logits.cpu().detach().numpy()[0], beam_width=100).text

#     input_values = processor(
#         batch["speech"],
#         sampling_rate=batch["sampling_rate"],
#         return_tensors="pt"
#     ).input_values.to("cuda")

#     with torch.no_grad():
#       logits = model(input_values).logits

#     pred_ids = torch.argmax(logits, dim=-1)
#     batch["pred_str"] = processor.batch_decode(pred_ids)[0]

    return batch

# def map2res(batch):
#     model.to('cuda')
#     input_values = processor(
#         batch["speech"],
#         sampling_rate=batch["sampling_rate"],
#         return_tensors="pt"
#     ).input_values.to("cuda")

#     with torch.no_grad():
#       logits = model(input_values).logits

#     pred_ids = torch.argmax(logits, dim=-1)
#     batch["pred_str"] = processor.batch_decode(pred_ids)[0]

#     return batch

In [ ]:
# torch.cuda.empty_cache()
results = data[:1000].map(map_to_result)
# results = combined.map(map_to_result)

AttributeError: ignored

In [ ]:
gc.collect()

39

In [ ]:
results = Dataset.from_dict(data[:1000]).map(map_to_result)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
results['pred_str']

['cái đèn tranh trong nhà giữ đồ trường sa có còn không ấy nhở đi kiểm tra ngay nhá',
 'bật màn ứng lên có rời ma hụi',
 'bạn ơi chán quá còn cái gì nghe không mở radio giúp mình với',
 'máy tính ơi tưới cây đủ rồi đóng giúp tôi cái vòi tưới với',
 'em ơi giảm độ sáng đèn tường sướng cho anh nhá',
 'giảm cho anh cái bóng đứng lo lên',
 'phòng này đầy mùi thôi tăng cái quạt hút mùi lên xem nào',
 'bạn mở giúp mình cái tủ lạnh với',
 'đóng cái màn cuốn trong phòng của long lại nhá',
 'chào bạn bạn có thể đóng van nước trong nhà tắm hộ mình được không',
 'kiểm tra xem cái đèn học tắt chưa nhá',
 'em gái ơi kiểm tra giúp chị cái bóng đứng ở ban công ba nhé',
 'sao tối thế bật điện lên cho anh với',
 'hình như có gì chưa đúng lắm tắt điện hộ mình với',
 'đóng van nước hộ mình với',
 'mình chuẩn bị đi ngủ rồi tắt giúp mình cái đèn tròn với',
 'chị ơi tăng hộ em cái bóng làm việc với',
 'nhà thông minh tăng cho mình bình nóng lạnh',
 'em ơi đóng giúp anh cái vòi nước với',
 'ở phòng chơi gam 

In [ ]:
%cd

/root


In [ ]:
from infer import reformat_normed_term
reformat_normed_term(list("h"), 'giờ')

['h']

In [ ]:
bias_list = ['phút | phút']
input_str = ['kiểm tra cho tôi cái bóng làm việc ở đầu hè vào lúc mười một giờ hai mươi hai phút']

In [ ]:
print(infer(input_str, bias_list))

['kiểm tra cho tôi cái bóng làm việc ở đầu hè vào lúc 11h22 phút']


In [ ]:
# combined = concatenate_datasets([data['valid'], data['test']])

In [ ]:
# model.to('cuda')

In [ ]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))

Test WER: 0.103


In [ ]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))

,target_text,pred_str
0,kiểm tra cho tôi cái bóng làm việc ở đầu hè vào lúc 11 giờ 22 phút,kiểm tra cho tôi cái bóng làm việc ở đầu hè vào lúc mười một giờ hai mươi hai phút
1,tắt dùm chị cái bóng tuýp cái,tắt giùm chị cái bóng tuýp cái
2,uây mưa to quá bạn đóng cho mình cái màn cửa vào nhá,ơi mưa to quá bạn đóng cho mình cái màn cửa vào nhá
3,em ơi kiểm tra giúp anh cái máy in với,em ơi kiểm tra giúp anh cái máy in với
4,kiểm tra đèn ốp tường em ơi,kiểm tra đèn ốp tường em ơi


In [ ]:
# del model, processor

In [ ]:
# %cd /content/drive/MyDrive/SLU_hackathon/

In [ ]:
# results.to_json("pred_str.jsonl")

In [ ]:
# test_results = load_dataset("json", data_files="pred_str.jsonl")

In [ ]:
# test_results['train']['pred_str']

In [ ]:
# model.to("cuda")
# input_values = processor(data["test"][0]["speech"], sampling_rate=data["test"][0]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

# with torch.no_grad():
#   logits = model(input_values).logits

# pred_ids = torch.argmax(logits, dim=-1)

# # convert ids to tokens
# " ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))